Title: "DSCI 6607: Programmatic Data Analysis with R and Python"

Subtitle: "Pandas III: Data Wrangling"

Author: "Dr. Armin Hatefi"

Date: "Friday, November 15, 2024"

**Note:** This content is protected and may not be shared, uploaded, or distributed.

# Outline
- Hierarchical Indexing
- Summary Statistics by Level
- Merge, Join and Concat DataFrames
- GroupBy, aggregation, apply

In [1]:
import numpy as np
import pandas as pd

## Hierarchical Indexing

`Hierarchical indexing` is an important feature of pandas that enables you to have multiple (two or more) index levels on an axis. Another way of thinking about it is that it provides a way for you to work with higher dimensional data in a lower dimensional form. 

In [2]:
data = pd.Series(np.random.uniform(size=9),
                 index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.991895
   2    0.039974
   3    0.409067
b  1    0.703479
   3    0.166348
c  1    0.509335
   2    0.203312
d  2    0.906506
   3    0.996194
dtype: float64

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

With a hierarchically indexed object, so-called partial indexing is possible, enabling you to concisely select subsets of the data:

In [4]:
data["b"]

1    0.703479
3    0.166348
dtype: float64

In [5]:
data["b":"c"]

b  1    0.703479
   3    0.166348
c  1    0.509335
   2    0.203312
dtype: float64

In [6]:
data.loc[["b", "d"]]

b  1    0.703479
   3    0.166348
d  2    0.906506
   3    0.996194
dtype: float64

Selection is even possible from an “inner” level. Here I select all of the values having the value 2 from the second index level:

In [7]:
data.loc[:, 2]

a    0.039974
c    0.203312
d    0.906506
dtype: float64

- Hierarchical indexing plays an important role  in group-based operations like forming a pivot table. 
- For example, you can rearrange this data into a DataFrame using its unstack method:

In [8]:
data.unstack()

,1,2,3
a,0.991895,0.039974,0.409067
b,0.703479,NaN,0.166348
c,0.509335,0.203312,NaN
d,NaN,0.906506,0.996194


In [9]:
data.unstack().stack()

a  1    0.991895
   2    0.039974
   3    0.409067
b  1    0.703479
   3    0.166348
c  1    0.509335
   2    0.203312
d  2    0.906506
   3    0.996194
dtype: float64

- With a DataFrame, either axis can have a hierarchical index.
- The hierarchical levels can have names

In [10]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[["a", "a", "b", "b"], [1, 2, 1, 2]],
                     columns=[["Ohio", "Ohio", "Colorado"],
                              ["Green", "Red", "Green"]])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [11]:
frame.index.names = ["key1", "key2"]
frame.columns.names = ["state", "color"]
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

You can see how many levels an index has by accessing its `nlevels` attribute:

In [12]:
frame.index.nlevels

2

With partial column indexing you can similarly select groups of columns:

In [13]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

- At times, you may need to rearrange the order of the levels on an axis.

In [14]:
frame.swaplevel(axis=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [15]:
frame.swaplevel(axis=1)

color     Green  Red    Green
state      Ohio Ohio Colorado
key1 key2                    
a    1        0    1        2
     2        3    4        5
b    1        6    7        8
     2        9   10       11

In [16]:
frame.swaplevel(axis=1)['Green']

state      Ohio  Colorado
key1 key2                
a    1        0         2
     2        3         5
b    1        6         8
     2        9        11

- `sort_index` by default sorts the data  using all the index levels, but you can choose to use only a single level or a subset of levels to sort by passing the level argument.

In [17]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [18]:
frame.sort_index(axis=1,level=1)

state     Colorado  Ohio    
color        Green Green Red
key1 key2                   
a    1           2     0   1
     2           5     3   4
b    1           8     6   7
     2          11     9  10

## Summary Statistics by Level

Many descriptive and summary statistics on DataFrame and Series have a level option in which you can specify the level you want to aggregate by on a particular axis.

In [19]:
frame.groupby(level="key2").sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [20]:
frame.groupby(level="color", axis="columns").sum()

/var/folders/2d/9508nsz51vx2htl27dgrdbfc0000gn/T/ipykernel_97242/775557097.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  frame.groupby(level="color", axis="columns").sum()


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

We will discuss `groupby` in much more detail later

- It’s not unusual to want to use one or more columns from a DataFrame as the row index; 
- alternatively, you may wish to move the row index into the DataFrame’s columns. 

In [21]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                      "c": ["one", "one", "one", "two", "two",
                            "two", "two"],
                      "d": [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [22]:
frame2 = frame.set_index(["c", "d"])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [23]:
frame.set_index(["c", "d"], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

`reset_index`, on the other hand, does the opposite of `set_index`; the hierarchical index levels are moved into the columns:

In [24]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## Merging DataFrames
Merge or join operations combine datasets by linking rows using one or more keys.

In [25]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7))})
df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": pd.Series(range(3))})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [26]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


- This is an example of join; the data in `df1` has multiple rows labeled a and b, whereas `df2` has only one row for each value in the key column

In [27]:
pd.merge(df1, df2, on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [28]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7))})
df4 = pd.DataFrame({"rkey": ["a", "b", "d"],
                    "data2": pd.Series(range(3))})
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [29]:
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


- By default, `pandas.merge` does an "inner" join; the keys in the result are the intersection, or the common set found in both tables. The "outer" join takes the union of the keys,

In [30]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey",how="outer")

,lkey,data1,rkey,data2
0,a,2.0,a,0.0
1,a,4.0,a,0.0
2,a,5.0,a,0.0
3,b,0.0,b,1.0
4,b,1.0,b,1.0
5,b,6.0,b,1.0
6,c,3.0,NaN,NaN
7,NaN,NaN,d,2.0


In [31]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                     "key2": ["one", "two", "one"],
                     "lval": pd.Series([1, 2, 3])})
right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                      "key2": ["one", "one", "one", "two"],
                      "rval": pd.Series([4, 5, 6, 7])})
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [32]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [33]:
pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,bar,one,3.0,6.0
1,bar,two,NaN,7.0
2,foo,one,1.0,4.0
3,foo,one,1.0,5.0
4,foo,two,2.0,NaN


DataFrame has a `join` instance method to simplify merging by index. It can also be used to combine many DataFrame objects having the same or similar indexes but nonoverlapping columns.

In [34]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=["a", "c", "e"],
                     columns=["Ohio", "Nevada"])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=["b", "c", "d", "e"],
                      columns=["Missouri", "Alabama"])
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [35]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [36]:
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


Another kind of data combination operation is  `pandas.concat` to glue together objects:

In [37]:
s1 = pd.Series([0, 1], index=["a", "b"])
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"])
s3 = pd.Series([5, 6], index=["f", "g"])

In [38]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [39]:
pd.concat([s1, s2, s3], axis="columns")

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In the case of DataFrames where the row index does not contain any relevant data:

In [40]:
df1 = pd.DataFrame(np.random.standard_normal((3, 4)),
                   columns=["a", "b", "c", "d"])
df2 = pd.DataFrame(np.random.standard_normal((2, 3)),
                   columns=["b", "d", "a"])
df1

,a,b,c,d
0,-0.883043,-0.358822,-0.383893,-0.820144
1,-0.741200,-1.501935,0.744667,-0.078736
2,-0.122935,-0.268260,-2.150817,-1.693707


In [41]:
df2

,b,d,a
0,0.230684,0.376067,0.748645
1,0.044353,-0.720276,0.152075


In [42]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,-0.883043,-0.358822,-0.383893,-0.820144
1,-0.741200,-1.501935,0.744667,-0.078736
2,-0.122935,-0.268260,-2.150817,-1.693707
3,0.748645,0.230684,NaN,0.376067
4,0.152075,0.044353,NaN,-0.720276


For more details, see the pandas documentions about `merge` [(here)](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html), `join`[(here)](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html) and `concat` [(here)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html)

## GroupBy

In [43]:
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1],dtype="Int64"),
                   "data1" : np.random.standard_normal(7),
                   "data2" : np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,1.307656,-0.893088
1,a,2,-0.694337,0.284094
2,None,1,0.506543,1.172608
3,b,2,-1.318434,-0.283634
4,b,1,-0.734321,-0.460683
5,a,<NA>,-1.298542,-0.705260
6,None,1,-1.909232,-0.006852


Suppose you wanted to compute the mean of the `data1` column using the labels from `key1`.

In [44]:
grouped = df["data1"].groupby(df["key1"])
grouped.mean()

key1
a   -0.228408
b   -1.026377
Name: data1, dtype: float64

In [45]:
grouped = df.groupby(df["key1"])
grouped.sum()

,key2,data1,data2
key1,,,
a,3,-0.685223,-1.314254
b,3,-2.052755,-0.744317


In [46]:
sums =  df['data1'].groupby([df["key1"], df["key2"]]).sum()
sums

key1  key2
a     1       1.307656
      2      -0.694337
b     1      -0.734321
      2      -1.318434
Name: data1, dtype: float64

In [47]:
sums.unstack()

key2,1,2
key1,,
a,1.307656,-0.694337
b,-0.734321,-1.318434


- A generally useful GroupBy method is `size`, which returns a Series containing group sizes:

In [48]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [49]:
df.groupby("key1", dropna=False).size()

key1
a      3
b      2
NaN    2
dtype: int64

A group function similar in spirit to size is `count`, which computes the number of nonnull values in each group:

In [50]:
df

,key1,key2,data1,data2
0,a,1,1.307656,-0.893088
1,a,2,-0.694337,0.284094
2,None,1,0.506543,1.172608
3,b,2,-1.318434,-0.283634
4,b,1,-0.734321,-0.460683
5,a,<NA>,-1.298542,-0.705260
6,None,1,-1.909232,-0.006852


In [51]:
df.groupby("key1", dropna=False).count()

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2
NaN,2,2,2


In [53]:
tips = pd.read_csv("tips.csv")

In [54]:
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


Here we passed a list of aggregation functions to `agg` to evaluate independently on the data groups.

In [ ]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips

In [ ]:
grouped = tips.groupby(["day", "smoker"])

In [ ]:
functions = ["count", "mean", "max","std"]
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

In [ ]:
grouped[["tip_pct", "total_bill"]].agg('mean')

The most general-purpose GroupBy method is `apply`,

**Example:** From tipping dataset, suppose you want to select the top five `tip_pct` values by group.
- First, write a function that selects the rows with the largest values in a particular column

In [ ]:
def top(df, n=5, column="tip_pct"): 
    return df.sort_values(by=column, ascending=False)[:n]

In [ ]:
top(tips,n=6)

In [ ]:
tips.groupby('smoker').apply(top,n=3)

In [ ]:
tips.groupby('smoker').apply(top,n=3,column="total_bill")

In [ ]:
tips.groupby('smoker')['tip_pct'].describe()

In [ ]:
f = lambda x: x.describe()

In [ ]:
tips.groupby('smoker')['tip_pct'].apply(f)

In [ ]:
tips.groupby('smoker')['tip_pct'].apply(f).unstack('smoker')

In [ ]:
pd.crosstab([tips.smoker,tips.day],tips.smoker,margins=True)

In [55]:
df1

,a,b,c,d
0,-0.883043,-0.358822,-0.383893,-0.820144
1,-0.741200,-1.501935,0.744667,-0.078736
2,-0.122935,-0.268260,-2.150817,-1.693707


In [56]:
df2

,b,d,a
0,0.230684,0.376067,0.748645
1,0.044353,-0.720276,0.152075


In [57]:
df_concat = pd.concat([df1, df2], axis=0, ignore_index=True)
df_concat

,a,b,c,d
0,-0.883043,-0.358822,-0.383893,-0.820144
1,-0.741200,-1.501935,0.744667,-0.078736
2,-0.122935,-0.268260,-2.150817,-1.693707
3,0.748645,0.230684,NaN,0.376067
4,0.152075,0.044353,NaN,-0.720276
